Copyright 2021 The Balloon Learning Environment Authors.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
# @title Imports
import collections
import json

from google.colab import files
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import HTML

In [ ]:
# @title Upload Data
uploaded_files = files.upload()
dataframes = dict()

for name, data in uploaded_files.items():
  name = name.rsplit('.', maxsplit=1)[0]
  json_data = json.loads(data)
  dataframes[name] = pd.DataFrame.from_dict(json_data)

In [ ]:
# @title Print Aggregate results
aggregate_data = collections.defaultdict(list)
for name, df in dataframes.items():
  aggregate_data['num episodes'].append(len(df.out_of_power))
  aggregate_data['out of power'].append(df.out_of_power.sum())
  aggregate_data['zeropressure'].append(df.zeropressure.sum())
  aggregate_data['envelope burst'].append(df.envelope_burst.sum())

  finished_runs = df.loc[df.out_of_power == False]
  finished_runs = finished_runs.loc[finished_runs.zeropressure == False]
  finished_runs = finished_runs.loc[finished_runs.envelope_burst == False]
  aggregate_data['mean cumulative reward (finished episodes)'].append(
      finished_runs.cumulative_reward.mean())
  aggregate_data['mean TWR50 (finished episodes)'].append(
      finished_runs.time_within_radius.mean())
  aggregate_data['mean cumulative reward (all episodes)'].append(
      df.cumulative_reward.mean())
  aggregate_data['mean TWR50 (all episodes)'].append(
      df.time_within_radius.mean())

df = pd.DataFrame(aggregate_data)
df.index = dataframes.keys()

# This is a little hacky, but it works 🤷‍♂️ 
html = df.to_html()
html += "<style>th {max-width: 150px}</style>"
HTML(html)